<a href="https://colab.research.google.com/github/Jarvis-BITS/midodrine-mimic-iv/blob/demo/notebook/04_tableone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
!pip install causalml
!pip install psmpy
!pip install pysmatch
!pip install tableone

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tableone import TableOne
from causalml import match
from pysmatch.Matcher import Matcher
from psmpy import PsmPy
from psmpy.functions import cohenD
from psmpy.plotting import *
from sklearn.linear_model import LogisticRegression as lr
import seaborn as sns
sns.set(rc={'figure.figsize':(16,10)}, font_scale=1.3)

import warnings
warnings.filterwarnings('ignore')

In [5]:
prop_score = pd.read_csv('ps.csv')

In [19]:
prop_score

,ps
0,0.083051
1,0.137752
2,0.100607
3,0.102545
4,0.039634
...,...
35295,0.003274
35296,0.034254
35297,0.007056
35298,0.004737


In [7]:
cohort_table = pd.read_csv('table_cohort.csv')

In [11]:
cohort_table

,stay_id,hadm_id,subject_id,age,Gender,ethnicity,midodrine,avg_doses_per_24_hrs,MICU,adm_weekday,...,resp_rate_mean,temperature_mean,spo2_mean,glucose_mean,sedative_use,ventilation_use,vasopressor_use,mortality,mort_28_day,admit_hour
0,36014877,27162817,11861017,87,0,WHITE,1,1.595745,0,1,...,17.600000,36.416667,100.000000,247.375000,1.0,1.0,1,0,0,15
1,32272859,25963696,11259141,60,1,WHITE,1,1.633333,0,7,...,17.333333,36.898333,97.259259,120.200000,1.0,1.0,1,0,0,18
2,34023828,21782431,18010960,59,0,UNKNOWN,1,1.589041,0,1,...,15.392857,36.786667,96.629630,98.000000,1.0,1.0,1,0,0,15
3,37798199,29764966,10926537,42,1,BLACK/AFRICAN AMERICAN,1,1.568627,0,7,...,17.240000,37.142857,99.791667,95.000000,1.0,1.0,1,0,0,17
4,32309766,29272306,15703353,59,1,WHITE,1,1.517241,0,6,...,21.740741,37.372000,98.178571,136.800000,1.0,1.0,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35295,39955170,24679700,12799579,55,1,WHITE,0,1.464286,1,7,...,24.148148,37.325000,94.115385,148.333333,0.0,1.0,1,0,0,19
35296,39977793,20968809,13886051,66,1,WHITE,0,1.407407,1,3,...,14.400000,36.646364,97.205882,96.333333,0.0,1.0,1,1,1,19
35297,39980385,21253505,11392990,73,0,WHITE,0,1.592593,1,5,...,19.605263,36.835000,93.605263,203.200000,0.0,1.0,1,1,0,6
35298,39985296,29295929,19632565,59,1,WHITE,0,1.782609,1,2,...,25.692308,36.603333,96.296296,173.400000,0.0,1.0,1,1,1,15


In [17]:
cohort_table.columns

Index(['stay_id', 'hadm_id', 'subject_id', 'age', 'Gender', 'ethnicity',
       'midodrine', 'avg_doses_per_24_hrs', 'MICU', 'adm_weekday', 'weight',
       'sapsii', 'charlson_index', 'CPD_or_COPD_rate', 'Maligancy_rate',
       'mild_liver_rate', 'severe_liver_rate', 'Renal_rate', 'CAD_rate',
       'AFIB_rate', 'hemoglobin', 'platelet', 'wbc', 'sodium', 'potassium',
       'bicarbonate', 'chloride', 'bun', 'creatinine', 'lactate', 'ph', 'po2',
       'pco2', 'bnp', 'troponin', 'creatinine_kinase', 'heart_rate_mean',
       'sbp_mean', 'dbp_mean', 'mbp_mean', 'resp_rate_mean',
       'temperature_mean', 'spo2_mean', 'glucose_mean', 'sedative_use',
       'ventilation_use', 'vasopressor_use', 'mortality', 'mort_28_day',
       'admit_hour'],
      dtype='object')

## Creating Population Table
Tom J Pollard, Alistair E W Johnson, Jesse D Raffa, Roger G Mark; tableone: An open source Python package for producing summary statistics for research papers, JAMIA Open, https://doi.org/10.1093/jamiaopen/ooy012

In [33]:
table_columns = ['midodrine', 'age', 'Gender', 'MICU', 'weight',
       'sapsii', 'charlson_index', 'CPD_or_COPD_rate', 'Maligancy_rate',
       'mild_liver_rate', 'severe_liver_rate', 'Renal_rate', 'CAD_rate',
       'AFIB_rate', 'hemoglobin', 'platelet', 'wbc', 'sodium', 'potassium',
       'bicarbonate', 'chloride', 'bun', 'creatinine', 'lactate', 'ph', 'po2',
       'pco2', 'bnp', 'troponin', 'creatinine_kinase', 'heart_rate_mean',
       'sbp_mean', 'dbp_mean', 'mbp_mean', 'resp_rate_mean',
       'temperature_mean', 'spo2_mean', 'glucose_mean', 'sedative_use', 
       'ventilation_use', 'mort_28_day', 'adm_weekday', 'ethnicity']

table_categorical = ['Gender', 'ethnicity', 'adm_weekday']

table_groupby = 'midodrine'

table_nonnormal = ['age', 'weight', 'sapsii', 'charlson_index', 'bicarbonate', 'bun',
             'creatinine', 'spo2_mean']

In [34]:
mytable = TableOne(cohort_table, columns=table_columns, categorical=table_categorical, groupby=table_groupby, nonnormal=table_nonnormal, pval=True)

In [40]:
print(mytable.tabulate(tablefmt = "fancy_grid"))

╒════════════════════════════════╤═══════════════════════════════╤═══════════╤══════════════════╤══════════════════╤══════════════════╤═══════════╕
│                                │                               │ Missing   │ Overall          │ 0                │ 1                │ P-Value   │
╞════════════════════════════════╪═══════════════════════════════╪═══════════╪══════════════════╪══════════════════╪══════════════════╪═══════════╡
│ n                              │                               │           │ 35300            │ 34490            │ 810              │           │
├────────────────────────────────┼───────────────────────────────┼───────────┼──────────────────┼──────────────────┼──────────────────┼───────────┤
│ age, median [Q1,Q3]            │                               │ 0         │ 64.0 [51.0,77.0] │ 64.0 [51.0,77.0] │ 63.0 [53.0,73.0] │ 0.210     │
├────────────────────────────────┼───────────────────────────────┼───────────┼──────────────────┼───────────────